In [1]:
from configure import GlobalVariable
gvar = GlobalVariable()
print gvar

loading data...
end of loading
end of init


In [2]:
%reload_ext autoreload
% autoreload 2

In [3]:
from zutil.config import Config
from data import QuestionDataset
import torch

model = torch.load('checkpoint.pt')['model']
model.eval()

# eval set
config_eval = Config('parameter.json', mode='inference')
dataset_eval = QuestionDataset(config_eval, gvar)

print config_eval

{u'n_classes': 1999, u'max_qlen': 55, u'max_epoch': 100, u'learning_rate': 0.001, u'mlp_hidden_size': 1024, u'rnn_hidden_size': 300, u'batch_size': 200, u'split_rate': 0.9, u'emb_dim': 256, u'nothing': u'_', u'mode': 'inference', u'question_type': u'title', u'model_type': u'baseline', u'gpu': True, u'rnn_hidden_layers': 3}


In [13]:
# get top5 results
import numpy as np
count = 0
results = np.zeros((len(dataset_eval), 5))
for questions, indices in dataset_eval.next_batch():
    count += 1
    if count % 300 == 0:
        print count
    output = model(questions)
    output= output.cpu().data.topk(5)[1].numpy()
    indices = indices.cpu().data.numpy().astype('int')
    results[indices] = output
results = results.astype('int')
print results[0:10]

300
600
900


In [33]:
# write result to file
with open('result.csv', 'w') as f:
    count = 0
    print results.shape[0]
    for i in range(results.shape[0]):
        answer = gvar.question_eval[i][0] + ',' 
        if results[i].any() == 0:
            ids = [topic[0] for topic in gvar.topics[0:5]]
            print i
        else:
            ids = [gvar.index2topic[topk] for topk in results[i]]
        answer += ','.join(ids) + '\n'
        f.write(answer)

217360
110081
166420
182280
194409
